# <center> Reconnaissance visuelle 

# TODO – Reconnaissance visuelle (Visual Recognition)

## 1. Préparation du dataset pour vision
- [ ] Vérifier que le sous-dataset WikiArt est chargé avec 400 images par style.
- [ ] Assurer la structure des dossiers : `train/val/test` avec sous-dossiers par `style`.
- [ ] Vérifier que les fonctions `load_image()` et `DATA_ROOT` fonctionnent correctement.
- [ ] Optionnel : filtrer images non “peinture” (statues, architecture…) pour observation qualitative.

## 2. Extraction de features
### 2.1. Contours et gradients
- [ ] Appliquer **détecteur de Sobel** sur chaque image.
- [ ] Calculer histogrammes pour chaque image :
  - Magnitude des gradients
  - Orientation des gradients
- [ ] Vérifier que les histogrammes différencient bien styles nets vs flous.

### 2.2. Descripteurs globaux
- [ ] Extraire **HOG** (Histogram of Oriented Gradients) pour chaque image.
- [ ] Extraire descripteurs de couleur dans les espaces **Lab** et **HSV**.
- [ ] Normaliser et combiner toutes les features en un seul vecteur par image.
- [ ] Créer une fonction `extract_features(image)` réutilisable.

## 3. Classification SVM
- [ ] Séparer features en train / val / test.
- [ ] Normaliser / standardiser les features.
- [ ] Entraîner un **SVM avec noyau RBF** sur le train.
- [ ] Évaluer le modèle sur val et test :
  - Précision globale
  - F1-score par style
  - Matrices de confusion détaillées
- [ ] Identifier les confusions fréquentes et vérifier si elles correspondent à des confusions humaines.
- [ ] Identifier les features les plus discriminantes pour chaque style.

## 4. Analyse et visualisation
- [ ] Visualiser quelques images par style pour valider les features.
- [ ] Histogrammes :
  - Nombre d’images par style
  - Distribution des artistes par style (exclure “unknown”)
  - Distribution des genres par style (exclure “unknown”)
- [ ] Analyse qualitative des styles avec images non “peinture” (statues, architecture…).
- [ ] Étudier la proportion d’artistes communs entre splits et risques de fuite.
- [ ] Visualiser relations artistes ↔ styles et genres ↔ styles.
- [ ] Préparer graphiques clairs pour le rapport (histogrammes, exemples d’images).

## 5. Validation causale par ablation (facultatif / si FFT fait par l’autre personne)
- [ ] Créer versions du dataset :
  - Original
  - Passe-haut
  - Passe-bas
- [ ] Entraîner un ResNet18 pour chaque version.
- [ ] Comparer performances pour confirmer l’importance des signatures fréquentielles.

## Notes
- Toutes les fonctions d’extraction doivent être centralisées dans `src/dataset.py`.
- Préparer les notebooks pour visualisation et tests rapides sans écraser les fichiers originaux.
